In [1]:
import json
import numpy as np
import os
import time
import cv2
from typing import Any

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml
from torch.utils.data import DataLoader
from torchvision import transforms
from torchinfo import summary

from GaborNet import GaborConv2d
from dataset import DogsCatsDataset
from dataset import JAFFEDataset
from sklearn.model_selection import train_test_split
import splitfolders


In [7]:
train_dir = os.path.join("../jaffe")
train_list_file = os.listdir(train_dir)
print(len(train_list_file))

213


In [7]:
print(train_list_file)

['MK.AN3.127.tiff', 'MK.NE1.113.tiff', 'YM.AN3.63.tiff', 'MK.SA1.119.tiff', 'KA.SA.35.tiff', 'KM.FE.25.tiff', 'YM.NE2.50.tiff', 'TM.HA2.181.tiff', 'NA.FE2.218.tiff', 'YM.NE3.51.tiff', 'MK.DI1.128.tiff', 'YM.AN2.62.tiff', 'YM.HA2.53.tiff', 'NA.AN2.212.tiff', 'KM.SA5.13.tiff', 'UY.FE1.152.tiff', 'UY.AN2.147.tiff', 'KR.NE1.71.tiff', 'KL.SU.166.tiff', 'NA.HA3.204.tiff', 'KL.DI.170.tiff', 'KM.NE.1.tiff', 'KA.SU.38.tiff', 'KA.AN.41.tiff', 'TM.SA1.184.tiff', 'KA.AN.40.tiff', 'YM.DI3.66.tiff', 'TM.NE1.177.tiff', 'KR.HA2.75.tiff', 'UY.DI1.149.tiff', 'KL.DI.171.tiff', 'KM.SU3.16.tiff', 'NA.AN3.213.tiff', 'YM.FE2.68.tiff', 'KM.DI.22.tiff', 'NA.FE3.219.tiff', 'KR.SU3.82.tiff', 'NM.AN3.106.tiff', 'UY.NE3.136.tiff', 'YM.FE3.69.tiff', 'KM.FE.24.tiff', 'NA.DI2.215.tiff', 'KA.SA.34.tiff', 'MK.AN2.126.tiff', 'KA.NE.28.tiff', 'TM.AN2.191.tiff', 'YM.FE4.70.tiff', 'KL.FE.175.tiff', 'KL.AN.168.tiff', 'KM.HA.5.tiff', 'NM.AN1.104.tiff', 'KL.HA.159.tiff', 'NM.FE2.111.tiff', 'UY.NE1.134.tiff', 'MK.SA2.120.tiff'

In [8]:
train_set_tiff, test_set_tiff = train_test_split(train_list_file, random_state=123, test_size=0.1)
print(len(train_set_tiff), len(test_set_tiff))

191 22


In [9]:
print(train_set_tiff)

['KA.SA.35.tiff', 'TM.DI3.195.tiff', 'NM.AN2.105.tiff', 'KM.NE.1.tiff', 'KL.HA.158.tiff', 'NM.HA2.96.tiff', 'NM.FE1.110.tiff', 'NM.SA2.99.tiff', 'KA.DI.43.tiff', 'UY.NE1.134.tiff', 'YM.SA1.55.tiff', 'TM.SU2.188.tiff', 'NA.SA2.206.tiff', 'KL.SU.165.tiff', 'MK.DI1.128.tiff', 'NA.FE1.217.tiff', 'YM.HA1.52.tiff', 'YM.DI3.66.tiff', 'YM.AN2.62.tiff', 'UY.AN1.146.tiff', 'NA.NE3.201.tiff', 'TM.SU1.187.tiff', 'UY.SA3.142.tiff', 'KL.AN.167.tiff', 'KR.AN1.83.tiff', 'KM.HA.7.tiff', 'NA.AN1.211.tiff', 'KR.NE2.72.tiff', 'KA.NE.26.tiff', 'NM.AN3.106.tiff', 'NM.AN1.104.tiff', 'YM.SA3.57.tiff', 'NM.DI3.109.tiff', 'MK.FE3.133.tiff', 'NA.NE1.199.tiff', 'TM.SA1.184.tiff', 'KL.NE.155.tiff', 'UY.HA3.139.tiff', 'YM.FE2.68.tiff', 'KA.AN.41.tiff', 'TM.SA2.185.tiff', 'MK.FE1.131.tiff', 'NA.DI2.215.tiff', 'KL.DI.172.tiff', 'KM.SA3.11.tiff', 'KR.FE3.91.tiff', 'NA.HA2.203.tiff', 'KR.DI3.88.tiff', 'MK.DI3.130.tiff', 'KR.FE2.90.tiff', 'TM.NE2.178.tiff', 'YM.SU2.59.tiff', 'NA.FE2.218.tiff', 'UY.NE2.135.tiff', 'UY.SA2

In [10]:
print(test_set_tiff)

['NA.SA3.207.tiff', 'NM.NE3.94.tiff', 'KM.NE2.2.tiff', 'KL.SA.162.tiff', 'YM.DI2.65.tiff', 'NM.SU3.103.tiff', 'NM.SU1.101.tiff', 'NM.DI1.107.tiff', 'KL.AN.169.tiff', 'KM.SU3.16.tiff', 'KA.SA.34.tiff', 'KA.NE.27.tiff', 'NM.NE1.92.tiff', 'KM.SU2.15.tiff', 'NA.HA3.204.tiff', 'KA.FE.47.tiff', 'KL.DI.170.tiff', 'NM.FE2.111.tiff', 'TM.FE3.198.tiff', 'NM.HA3.97.tiff', 'MK.SU3.124.tiff', 'UY.AN3.148.tiff']


In [11]:
angry=disgust=fear=happy=sad=suspecious=neutral = 0

for data in test_set_tiff:
    if 'AN' in data:
        angry+=1
    elif 'DI' in data:
        disgust+=1
    elif 'FE' in data:
        fear+=1
    elif 'HA' in data:
        happy+=1
    elif 'SA' in data:
        sad+=1
    elif 'SU' in data:
        suspecious+=1
    elif 'NE' in data:
        neutral+=1
    
print(angry, disgust, fear, happy, sad, suspecious, neutral)

2 3 3 2 3 5 4


In [12]:
angry=disgust=fear=happy=sad=suspecious=neutral = 0

#counter_dict = {'AN':0, 'DI':0, 'FE':0, 'HA':0, 'SA':0, 'SU':0, 'NE':0 }
for data in train_set_tiff:
    if 'AN' in data:
        angry+=1
    elif 'DI' in data:
        disgust+=1
    elif 'FE' in data:
        fear+=1
    elif 'HA' in data:
        happy+=1
    elif 'SA' in data:
        sad+=1
    elif 'SU' in data:
        suspecious+=1
    elif 'NE' in data:
        neutral+=1
    
print(angry, disgust, fear, happy, sad, suspecious, neutral)

28 26 29 29 28 25 26
